In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [4]:
root_path = '../../'

data = torch.load(f'{root_path}/data/pt/waves.pt')
data_medium = torch.load(f'{root_path}/data/pt/waves-medium-m3.pt')

/var/folders/d0/7p4xxyy97qbb5_f74yjfh9vw0000gp/T/ipykernel_13048/3921425368.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f'{root_path}/data/pt/waves

In [23]:
for index, (key, value) in enumerate(data.items()):
    print(index, key, '\t\t', value['features'].shape, value['target'].shape)

0 Ballybunnion 		 torch.Size([433, 5]) torch.Size([433])
1 Belmullet_Inner 		 torch.Size([433, 5]) torch.Size([433])
2 Belmullet_Outer 		 torch.Size([433, 5]) torch.Size([433])
3 Brandon_Bay 		 torch.Size([433, 5]) torch.Size([433])
4 Cork_SmartBuoy 		 torch.Size([433, 5]) torch.Size([433])
5 Finnis 		 torch.Size([433, 5]) torch.Size([433])
6 Killard 		 torch.Size([433, 5]) torch.Size([433])
7 M2 		 torch.Size([433, 5]) torch.Size([433])
8 M3 		 torch.Size([433, 5]) torch.Size([433])
9 M4 		 torch.Size([433, 5]) torch.Size([433])
10 M5 		 torch.Size([433, 5]) torch.Size([433])
11 M6 		 torch.Size([433, 5]) torch.Size([433])
12 Mace_Head 		 torch.Size([433, 5]) torch.Size([433])
13 South_Hunter 		 torch.Size([433, 5]) torch.Size([433])
14 Spiddal 		 torch.Size([433, 5]) torch.Size([433])
15 Splaugh 		 torch.Size([433, 5]) torch.Size([433])


In [34]:
train_test_data = []

for key, value in data.items():
    train_ratio = 0.8
    train_size = int(len(value['features']) * train_ratio)
    
    X_train, X_test = value['features'][:train_size], value['features'][train_size:]
    y_train, y_test = value['target'][:train_size], value['target'][train_size:]
    
    train_test_data.append({
        'name': key,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test
    })

In [35]:
for i in train_test_data:
    print(i['name'], '\t\t', i['X_train'].shape, i['X_test'].shape, i['y_train'].shape, i['y_test'].shape) 

Ballybunnion 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
Belmullet_Inner 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
Belmullet_Outer 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
Brandon_Bay 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
Cork_SmartBuoy 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
Finnis 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
Killard 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
M2 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
M3 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
M4 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
M5 		 torch.Size([346, 5]) torch.Size([87, 5]) torch.Size([346]) torch.Size([87])
M6 		 torch.Size([346, 5]) torch

### start with a simple linear model treating the data as tabular

In [38]:
class SimpleLinear(nn.Module):
    def __init__(self, input_dim, output_dim = 1):
        super(SimpleLinear, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### n-step ahead prediction using the same linear model

In [ ]:
n = 12  # n-step ahead prediction
runs = 10
num_epochs = 500
learning_rate = 0.001

for station in train_test_data:
    runs = 10
    total_avg_loss = 0

    X_train = station['X_train'][:n * -1]
    X_test = station['X_test'][:n * -1]
    
    y_train = station['y_train'][n:]
    y_test = station['y_test'][n:]
    
    model = MLP(X_train.shape[1], 32)

    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 1000

    for run in range(runs):

        for epoch in range(num_epochs):
            model.train()
            outputs = model(X_train).squeeze()
            
            loss = criterion(outputs, y_train)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred = model(X_test).squeeze()
            loss = criterion(y_pred, y_test)
            # print(f'Final Loss: {loss.item():.4f} meters')
            total_avg_loss += loss.item()

    print()
    station_name = station['name']
    print(f'Average Loss ({runs} runs) for station {station_name}: {total_avg_loss / runs:.4f} meters')


Average Loss (10 runs) for station Ballybunnion: 0.6738 meters

Average Loss (10 runs) for station Belmullet_Inner: 1.2132 meters

Average Loss (10 runs) for station Belmullet_Outer: 1.5384 meters

Average Loss (10 runs) for station Brandon_Bay: 0.3272 meters

Average Loss (10 runs) for station Cork_SmartBuoy: 2.1586 meters

Average Loss (10 runs) for station Finnis: 1.0010 meters

Average Loss (10 runs) for station Killard: 0.8424 meters

Average Loss (10 runs) for station M2: 1.1234 meters

Average Loss (10 runs) for station M3: 0.9604 meters

Average Loss (10 runs) for station M4: 1.8894 meters

Average Loss (10 runs) for station M5: 1.8926 meters

Average Loss (10 runs) for station M6: 2.4110 meters

Average Loss (10 runs) for station Mace_Head: 0.7994 meters

Average Loss (10 runs) for station South_Hunter: 1.5795 meters

Average Loss (10 runs) for station Spiddal: 0.6131 meters

Average Loss (10 runs) for station Splaugh: 1.1501 meters


### Using the Medium dataset 

In [38]:
Xm_train, Xm_test, ym_train, ym_test = train_test_split(Xm, ym, test_size=0.2, random_state=42)

tabular prediction

In [40]:
runs = 10
total_avg_loss = 0

# model = SimpleLinear(Xm_train.shape[1])
model = MLP(Xm_train.shape[1], 32)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 100

for run in range(runs):

    for epoch in range(num_epochs):
        model.train()
        outputs = model(Xm_train).squeeze()
        
        loss = criterion(outputs, ym_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        y_pred = model(Xm_test).squeeze()
        loss = criterion(y_pred, ym_test)
        # print(f'Final Loss: {loss.item():.4f} meters')
        total_avg_loss += loss.item()
        
print()
print(f'Average Loss ({runs} runs): {total_avg_loss / runs:.4f} meters')


Average Loss (10 runs): 0.9412 meters


n-step ahead prediction

In [74]:
n = 24 # each step is 1 hour

Xmn = Xm[:n * -1]
ymn = ym[n:]

train_size = int(0.8 * len(Xmn))

Xm_train, Xm_test = Xmn[:train_size], Xmn[train_size:]
ym_train, ym_test = ymn[:train_size], ymn[train_size:]

print(Xm_train.shape, ym_train.shape)
print(Xm_test.shape, ym_test.shape)

torch.Size([89503, 8]) torch.Size([89503])
torch.Size([22376, 8]) torch.Size([22376])


In [75]:
runs = 10
total_avg_loss = 0

# model_mn = SimpleLinear(Xm_train.shape[1])
model_mn = MLP(Xm_train.shape[1], 32)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model_mn.parameters(), lr=0.01)

num_epochs = 100

for run in range(runs):

    for epoch in range(num_epochs):
        model_mn.train()
        outputs = model_mn(Xm_train).squeeze()
        
        loss = criterion(outputs, ym_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model_mn.eval()
    with torch.no_grad():
        y_pred = model_mn(Xm_test).squeeze()
        loss = criterion(y_pred, ym_test)
        # print(f'Final Loss: {loss.item():.4f} meters')
        total_avg_loss += loss.item()

print()
print(f'Average Loss ({runs} runs): {total_avg_loss / runs:.4f} meters')


Average Loss (10 runs): 1.1371 meters


NC 41025 data

In [841]:
dataNC = torch.load('../data/pt/waves-41025.pt')

/var/folders/d0/7p4xxyy97qbb5_f74yjfh9vw0000gp/T/ipykernel_8788/1621255091.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataNC = torch.load('../data/pt/waves-41025.pt

In [842]:
Xnc = dataNC['features']
ync = dataNC['target']

In [843]:
X_train, X_test, y_train, y_test = train_test_split(Xnc, ync, test_size=0.2, random_state=42)

In [844]:
n = 6 # each step is 10 minutes

Xn = Xnc[:n * -1]
yn = ync[n:]

train_size = int(0.8 * len(Xn))

X_train, X_test = Xn[:train_size], Xn[train_size:]
y_train, y_test = yn[:train_size], yn[train_size:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([41784, 9]) torch.Size([41784])
torch.Size([10447, 9]) torch.Size([10447])


In [851]:
modelNC = SimpleLinear(X_train.shape[1])

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(modelNC.parameters(), lr=0.01)

num_epochs = 500

for epoch in range(num_epochs):
    modelNC.train()
    outputs = modelNC(X_train).squeeze()
    
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # print epoch
    if epoch % (num_epochs // 10) == 0:
        print(f'Epoch {epoch + 1:03d}: {loss.item():.4f} meters')

Epoch 001: 647.7593 meters
Epoch 051: 14.9001 meters
Epoch 101: 6.6098 meters
Epoch 151: 2.6699 meters
Epoch 201: 2.6367 meters
Epoch 251: 2.4404 meters
Epoch 301: 9.1784 meters
Epoch 351: 2.0719 meters
Epoch 401: 2.7606 meters
Epoch 451: 1.1368 meters


In [852]:
modelNC.eval()
with torch.no_grad():
    y_pred = modelNC(X_test).squeeze()
    loss = criterion(y_pred, y_test)
    print(f'Final Loss: {loss.item():.4f} meters')

Final Loss: 8.6745 meters
